Based on https://github.com/MedMNIST/MedMNIST/blob/main/examples/getting_started.ipynb

Env preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/MLMI

/content/drive/MyDrive/MLMI


In [ ]:
!pip install medmnist torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 9.1 MB/s 
     |████████████████████████████████| 87 kB 8.6 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=618a3e685fc8e128b73ced794271df2e68d3beff6a9adcde8fc91d2fae08c7ff
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F
import copy
from sklearn.metrics import accuracy_score

import os


import medmnist
from medmnist import INFO, Evaluator

import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import torch
import random
np.random.seed(42)
random.seed(10)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(999)

In [ ]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.1.0 @ https://github.com/MedMNIST/MedMNIST/


Parameters

In [ ]:
data_flag = 'pathmnist'
download = True

NUM_EPOCHS = 100
BATCH_SIZE = 64
PATIENCE = 20
lr = 0.001
weight_decay = 0.001
number_of_res_blocks = 5

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])
task

'multi-class'

MedMNIST data, preprocessing, data loader


In [ ]:
from torch.utils.data import Dataset
class TissueDataset(Dataset):
    def __init__(self, images, labels, transform='None'):
        self.image = csv.drop(['label'], axis=1)
        self.label = csv['label'].to_numpy()
        self.transform = transform
    def __len__(self):
        return len(self.image)
    def __getitem__(self, index):
        img = self.image.iloc[index].to_numpy()
        img = img.reshape(28, 28, 3)
        img = Image.fromarray(np.uint8(img))
        if self.transform:
            img = self.transform(img)
        label = self.label[index]
        return img, label

In [ ]:
from sklearn.model_selection import train_test_split

csv_path = './hmnist_28_28_RGB.csv'
csv = pd.read_csv(csv_path)

X_train, X_test, y_train, y_test = train_test_split(csv.drop(['label'], axis=1), csv['label'].to_numpy(), test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_dataset_transfer = TissueDataset(X_train, y_train, data_transform)

transfer_train_loader = data.DataLoader(train_dataset_transfer, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

val_dataset_transfer = TissueDataset(X_test, y_test, data_transform)

transfer_val_loader = data.DataLoader(val_dataset_transfer, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

(4000, 2352) (1000, 2352)


In [ ]:
for i in range(10):
  image = csv.drop(['label'], axis=1)
  img = image.iloc[i].to_numpy()
  img = img.reshape(28, 28, 3)
  img = Image.fromarray(np.uint8(img))
  display(img)
  print(csv['label'][i])

print(len(csv['label']))

2


2


2


2


2


2


2


2


2


2
5000


In [ ]:
def create_labeled_subset(dataset, n_classes=9, per_class = 1000):
  label_buckets = [[] for _ in range(0, n_classes)]
  for img, label in dataset:
    label_buckets[label.item()].append((img, label))

  label_sub_bucket = [[] for _ in range(0, n_classes)]
  
  for i, bucket in enumerate(label_buckets):
    label_sub_bucket[i] = random.sample(bucket, per_class)

  balanced_labelled = []
  for b in label_sub_bucket:
    balanced_labelled.extend(b)

  return data.DataLoader(balanced_labelled, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
val_dataset = DataClass(split='val', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader= data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

labeled_loader = create_labeled_subset(train_dataset, per_class=100)

  0%|          | 0/205615438 [00:00<?, ?it/s]

Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz


In [ ]:
# visualization

train_dataset.montage(length=1)

In [ ]:
# montage

val_dataset.montage(length=20).save('original.png')

ResNet model definition


In [ ]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [ ]:
class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()      
        self.info = 'in ' +  str(in_channels) + ', out ' + str(out_channels) + ', stride ' + str(stride)
        if stride > 1 or in_channels != out_channels:
            # Add strides in the skip connection and zeros for the new channels.
            self.skip = Lambda(lambda x: F.pad(x[:, :, ::stride, ::stride], (0, 0, 0, 0, 0, out_channels - in_channels), mode="constant", value=0))
        else:
            self.skip = nn.Sequential()
 
        conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        bn1 = nn.BatchNorm2d(out_channels)
        conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding='same')
        bn2 = nn.BatchNorm2d(out_channels)
        self.l1 = nn.Sequential(conv1, bn1)
        self.l2 = nn.Sequential(conv2, bn2)

    def forward(self, input):
        skip = self.skip(input)
        x = self.l1(input)
        x = F.relu(x)
        x = self.l2(x)
        return F.relu(x + skip)

In [ ]:
class ResidualStack(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride, num_blocks):
        super().__init__()
        first = [ResidualBlock(in_channels, out_channels, stride)]
        rest = [ResidualBlock(out_channels, out_channels) for i in range(num_blocks - 1)]
        self.modules_list = nn.Sequential(*(first + rest))
        
    def forward(self, input):
        return self.modules_list(input)

In [ ]:
custom_res_net = nn.Sequential(
          nn.Conv2d(n_channels, 16, kernel_size=3, stride=1, padding='same', bias=False),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          ResidualStack(16, 16, 1, number_of_res_blocks),
          ResidualStack(16, 32, 2, number_of_res_blocks),
          ResidualStack(32, 64, 2, number_of_res_blocks),
          nn.AdaptiveAvgPool2d(1),
          Lambda(lambda x: x.squeeze()),
          nn.Linear(64, n_classes)
      )

Train and evaluate

In [ ]:
def initialize_weight(module):
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
    elif isinstance(module, nn.BatchNorm2d):
        nn.init.constant_(module.weight, 1)
        nn.init.constant_(module.bias, 0)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
def run_epoch(model, optimizer, dataloader, criterion):
  
    model.train()

    epoch_loss = 0.0
    epoch_acc = 0.0

    for inputs, targets in tqdm(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
        else:
            targets = targets.squeeze().long()

        loss = criterion(outputs, targets)

        # outputs = outputs.cpu()
        # y_score = torch.cat((y_score, outputs), 0)
        
        
        top1 = torch.argmax(outputs, dim=1).long()
        ncorrect = torch.sum(top1 == targets)
        epoch_loss += loss.item()
        epoch_acc += ncorrect.item()

        loss.backward()
        optimizer.step()

    epoch_loss /= len(dataloader.dataset)
    epoch_acc /= len(dataloader.dataset)

    return epoch_loss, epoch_acc

In [ ]:
best_model_weights = None

def fit(model, optimizer, lr_scheduler, task, loader, v_loader):

    if task == "multi-label, binary-class":
      criterion = nn.BCEWithLogitsLoss()
    else:
      criterion = nn.CrossEntropyLoss()
    
    best_acc = 0.0
    curr_patience = 0.0
    min_val_loss = 1.0

    for epoch in range(NUM_EPOCHS):

      train_loss, train_acc = run_epoch(model, optimizer, loader, criterion)
      lr_scheduler.step()
      print(f"Epoch {epoch + 1: >3}/{NUM_EPOCHS}, train loss: {train_loss:.2e}, train acc: {train_acc}")
      
      val_loss, val_acc = test(model, v_loader, 'val', criterion)
      print(f"Epoch {epoch + 1: >3}/{NUM_EPOCHS}, val loss: {val_loss:.2e}, val acc: {val_acc}")

      if val_loss >= min_val_loss:
        curr_patience += 1
        if curr_patience == PATIENCE:
          break
      else:
        curr_patience = 0
        min_val_loss = val_loss
        best_model_weights = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), 'ResNet.pt')

    model.load_state_dict(best_model_weights)

In [ ]:
def test(model, data_loader, mode, criterion):
    model.eval()
    epoch_acc = 0.0
    epoch_loss = 0.0

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
            else:
                targets = targets.squeeze().long()

            loss = criterion(outputs, targets)

            top1 = torch.argmax(outputs, dim=1).long()
            ncorrect = torch.sum(top1 == targets)
            epoch_loss += loss.item()
            epoch_acc += ncorrect.item()
        
        epoch_loss /= len(data_loader.dataset)
        epoch_acc /= len(data_loader.dataset)  
    
    return epoch_loss, epoch_acc

In [ ]:
if task == 'multi-label, binary-class':
  criterion = nn.BCEWithLogitsLoss()
else:
  criterion = nn.CrossEntropyLoss()

criterion

CrossEntropyLoss()

In [ ]:
res_net_18_transfer = nn.Sequential(
          nn.Conv2d(n_channels, 64, kernel_size=7, stride=1, padding='same', bias=False),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          ResidualStack(64, 64, 1, 6),
          ResidualStack(64, 128, 2, 8),
          ResidualStack(128, 256, 2, 12),
          ResidualStack(256, 512, 2, 6),
          nn.AdaptiveAvgPool2d(1),
          Lambda(lambda x: x.squeeze()),
          nn.Linear(512, n_classes)
      )
_ = res_net_18_transfer.to(device)
_ = res_net_18_transfer.apply(initialize_weight)

In [ ]:
optimizer = optim.Adam(res_net_18_transfer.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30], gamma=0.1)

In [ ]:
fit(res_net_18_transfer, optimizer, lr_scheduler, task, transfer_train_loader, transfer_val_loader) 

100%|██████████| 79/79 [00:21<00:00,  3.71it/s]

Epoch   1/100, train loss: 3.26e-02, train acc: 0.4992


Epoch   1/100, val loss: 1.80e-02, val acc: 0.6246


100%|██████████| 79/79 [00:21<00:00,  3.70it/s]

Epoch   2/100, train loss: 1.33e-02, train acc: 0.6896


...

Epoch  99/100, train loss: 1.70e-03, train acc: 0.973


Epoch  99/100, val loss: 2.13e-03, val acc: 0.9126


100%|██████████| 79/79 [00:23<00:00,  3.41it/s]

Epoch 100/100, train loss: 1.15e-03, train acc: 0.9818


Epoch 100/100, val loss: 4.09e-04, val acc: 0.9938


In [ ]:
test(res_net_18_transfer, test_loader, 'test', criterion)

(0.17095545659822342, 0.1817548746518106)

In [ ]:
res_net_18 = nn.Sequential(
          nn.Conv2d(n_channels, 64, kernel_size=7, stride=1, padding='same', bias=False),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          ResidualStack(64, 64, 1, 6),
          ResidualStack(64, 128, 2, 8),
          ResidualStack(128, 256, 2, 12),
          ResidualStack(256, 512, 2, 6),
          nn.AdaptiveAvgPool2d(1),
          Lambda(lambda x: x.squeeze()),
          nn.Linear(512, n_classes)
      )
_ = res_net_18.to(device)
res_net_18.load_state_dict(res_net_18_transfer.state_dict())

<All keys matched successfully>

In [ ]:
optimizer = optim.Adam(res_net_18.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30], gamma=0.1)

In [ ]:
fit(res_net_18, optimizer, lr_scheduler, task, labeled_loader, val_loader) 

100%|██████████| 15/15 [00:04<00:00,  3.60it/s]


Epoch   1/100, train loss: 4.58e-02, train acc: 0.3522222222222222
Epoch   1/100, val loss: 2.61e+00, val acc: 0.18922431027588965


100%|██████████| 15/15 [00:04<00:00,  3.62it/s]


Epoch   2/100, train loss: 3.10e-02, train acc: 0.43666666666666665
Epoch   2/100, val loss: 5.77e-02, val acc: 0.3354658136745302


100%|██████████| 15/15 [00:04<00:00,  3.63it/s]


...

Epoch  65/100, train loss: 1.08e-02, train acc: 0.8044444444444444
Epoch  65/100, val loss: 1.18e-02, val acc: 0.7337065173930428


100%|██████████| 15/15 [00:04<00:00,  3.63it/s]


Epoch  66/100, train loss: 1.09e-02, train acc: 0.8
Epoch  66/100, val loss: 1.23e-02, val acc: 0.7240103958416634


100%|██████████| 15/15 [00:04<00:00,  3.63it/s]


Epoch  67/100, train loss: 9.43e-03, train acc: 0.8033333333333333
Epoch  67/100, val loss: 1.19e-02, val acc: 0.7353058776489404


In [ ]:
test(res_net_18, test_loader, 'test', criterion)

(0.016571211661302945, 0.7530640668523677)

In [ ]:
res_net_18_without_transfer = nn.Sequential(
          nn.Conv2d(n_channels, 64, kernel_size=7, stride=1, padding='same', bias=False),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          ResidualStack(64, 64, 1, 6),
          ResidualStack(64, 128, 2, 8),
          ResidualStack(128, 256, 2, 12),
          ResidualStack(256, 512, 2, 6),
          nn.AdaptiveAvgPool2d(1),
          Lambda(lambda x: x.squeeze()),
          nn.Linear(512, n_classes)
      )
_ = res_net_18_without_transfer.to(device)
_ = res_net_18_without_transfer.apply(initialize_weight)

In [ ]:
optimizer = optim.Adam(res_net_18_without_transfer.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10, 30], gamma=0.1)

In [ ]:
fit(res_net_18_without_transfer, optimizer, lr_scheduler, task, labeled_loader, val_loader) 

100%|██████████| 15/15 [00:04<00:00,  3.62it/s]


Epoch   1/100, train loss: 9.53e-02, train acc: 0.24666666666666667
Epoch   1/100, val loss: 1.27e+01, val acc: 0.11875249900039984


100%|██████████| 15/15 [00:04<00:00,  3.61it/s]


Epoch   2/100, train loss: 3.35e-02, train acc: 0.45666666666666667
Epoch   2/100, val loss: 7.63e-02, val acc: 0.4239304278288685


100%|██████████| 15/15 [00:04<00:00,  3.62it/s]


...

Epoch  71/100, train loss: 1.00e-02, train acc: 0.8011111111111111
Epoch  71/100, val loss: 1.33e-02, val acc: 0.6941223510595762


100%|██████████| 15/15 [00:04<00:00,  3.62it/s]


Epoch  72/100, train loss: 9.90e-03, train acc: 0.8044444444444444
Epoch  72/100, val loss: 1.40e-02, val acc: 0.6901239504198321


100%|██████████| 15/15 [00:04<00:00,  3.61it/s]


Epoch  73/100, train loss: 1.16e-02, train acc: 0.7955555555555556
Epoch  73/100, val loss: 1.39e-02, val acc: 0.6855257896841264


In [ ]:
test(res_net_18_without_transfer, test_loader, 'test', criterion)

(0.016384430499462033, 0.7183844011142061)

In [ ]:
from skimage.util import montage as skimage_montage

csv_path = './hmnist_28_28_RGB.csv'
csv = pd.read_csv(csv_path)

In [ ]:
images = csv.drop(['label'], axis=1).to_numpy()
images = images.reshape(-1, 28, 28, 3)
# img = image.iloc[i].to_numpy()
# img = img.reshape(28, 28, 3)
# img = Image.fromarray(np.uint8(img))

length = 20
n_sel = length * length
sel = np.random.choice(len(images), size=n_sel, replace=False)

sel_img = images[sel]
# for img in sel_img:
#   img = img.reshape(28, 28, 3)
#   img = Image.fromarray(np.uint8(img))
#   display(img)
montage_arr = skimage_montage(sel_img, multichannel=True)
montage_img = Image.fromarray(np.uint8(montage_arr))

In [ ]:
display(montage_img)

In [ ]:
montage_img.save('transfer.png')